# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [67]:
# Import Python packages 
import pandas as pd
import cassandra
import re # Regular expression operations
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [68]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

file_path_list = []
    
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    
print(len(file_path_list))

/home/workspace
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [69]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# remove file if exists
os.remove("event_datafile_new.csv")

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [70]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [71]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [72]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [73]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




<hr />

### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

<b>Query: </b>`select artist, song, length from music_app_history WHERE sessionId=338 AND itemInSession=4` 

#### <b>Note:<b> The reason why I choose <i><b>sessionId</b></i> and <i><b>itemInSession</b></i> as a composite key is that either one of each columns isn't guarantee the uniqueness but the combination of two columns would do

In [89]:
# Create table
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [90]:
# Insert records to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [91]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, length from music_app_history WHERE sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


<hr />

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

<b>Query: </b> `select artist, song, firstName, lastName from music_by_user_history WHERE userId=10 AND sessionId=182`   

#### <b>Note:<b> The reason why I choose <i><b>userId</b></i> and <i><b>sessionId</b></i> as a composite key is that either one of each columns isn't guarantee the uniqueness but the combination of two columns would do. 
#### Also, I decide to add <i><b>item_in_session</b></i> as an additional partition key together with the <b>WITH CLUSTERING ORDER BY (item_in_session DESC)</b> to mention the descending order. Doing these will give me two benefits:
- <i><b>item_in_session</b></i> as an additional partition key will help sorting the results by item_in_session in ascending order in the case we didn't specified <b>item_in_session</b> 
    in the <b>WHERE</b> clause
- <b>WITH CLUSTERING ORDER BY (item_in_session DESC)</b> will return the results in descening order if we specified <b>item_in_session</b> in the <b>WHERE</b> clause
##### As I mentioned above, doing this will give me the more flexible way to query this table (by <b>item_in_session</b> in either ways - descending or ascending orders

In [92]:
# Create table
query = "CREATE TABLE IF NOT EXISTS music_by_user_history "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                 PRIMARY KEY ((userId, sessionId), itemInSession)) \
                 WITH CLUSTERING ORDER BY (itemInSession  DESC);"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [93]:
# Insert records to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_by_user_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [94]:
# Do a SELECT to verify that the data have been inserted into each table
query = "select artist, song, firstName, lastName from music_by_user_history WHERE userId=10 AND sessionId=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


<hr />

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

<b>Query: </b> `select firstName, lastName from music_by_song_history WHERE song="All Hands Against His Own"`

#### <b>Note:<b> The reason why I choose <i><b>userId</b></i> and <i><b>song</b></i> as a composite key is that both of these columns are more relevant to the query. We need all users who listened to the same song so these combination of two columns will suite as a primary key

In [95]:
# Create table
# Note: I add song & userId as a composite key since different users can listen to same song title
query = "CREATE TABLE IF NOT EXISTS music_by_song_history "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [96]:
# Insert records to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_by_song_history (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [97]:
# Do a SELECT to verify that the data have been inserted into each table
query = "select firstName, lastName from music_by_song_history  WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [98]:
query = "drop table IF EXISTS music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table IF EXISTS music_by_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table IF EXISTS music_by_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [99]:
session.shutdown()
cluster.shutdown()